<a href="https://colab.research.google.com/github/morrowbord/Spark/blob/main/credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Кредитный скоринг
При принятии решения о выдаче кредита или займа учитывается т.н. «Кредитный скоринг» — рейтинг платежеспособности клиента. ИИ на основе модели, которую просчитывает машинное обучение — в ней много параметров — возраст, зарплата, кредитная история, наличие недвижимости, автомобиля, судимости и других признаков, после обработки которых выносится положительное или отрицательное решение

In [ ]:
# # Импортируем библиотеки
# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'

# import warnings
# warnings.filterwarnings('ignore')
# import numpy as np
# import pandas as pd   
# import matplotlib.pyplot as plt
# import seaborn as sns
# import itertools

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

In [17]:
from pyspark.sql import SparkSession

In [18]:
# from pyspark.sql import functions as F
from pyspark.sql.types import *

In [19]:
spark=SparkSession.builder.appName("scoring").getOrCreate()

# spark.stop()

# Данные:
[скачать](https://drive.google.com/file/d/1MuAyZiIm3b_r-AgQSj78tsRPqZpvv_2s/view?usp=sharing)

**application_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   CODE_GENDER	Gender	
*   FLAG_OWN_CAR	Is there a car	
*   FLAG_OWN_REALTY	Is there a property	
*   CNT_CHILDREN	Number of children	
*   AMT_INCOME_TOTAL	Annual income	
*   NAME_INCOME_TYPE	Income category	
*   NAME_EDUCATION_TYPE	Education level	
*   NAME_FAMILY_STATUS	Marital status	
*   NAME_HOUSING_TYPE	Way of living	
*   DAYS_BIRTH	Birthday	Count backwards from current day (0), -1 means yesterday
*   DAYS_EMPLOYED	Start date of employment	Count backwards from current day(0). If positive, it means the person currently unemployed.
FLAG_MOBIL	Is there a mobile phone	
*   FLAG_WORK_PHONE	Is there a work phone	
*   FLAG_PHONE	Is there a phone	
*   FLAG_EMAIL	Is there an email	
*   OCCUPATION_TYPE	Occupation	
*   CNT_FAM_MEMBERS	Family size	

**credit_record.csv**
*   Feature name	Explanation	Remarks
*   ID	Client number	
*   MONTHS_BALANCE	Record month	The month of the extracted data is the starting point, backwards, 0 is the current month, -1 is the previous month, and so on
*   STATUS	Status	
   *   0: 1-29 days past due
   *   1: 30-59 days past due 
   *   2: 60-89 days overdue 
   *   3: 90-119 days overdue 
   *   4: 120-149 days overdue 
    *   5: Overdue or bad debts, write-offs for more than 150 days
    *   C: paid off that month X: No loan for the month


## Считываем данные

In [ ]:
# data = pd.read_csv("application_record.csv", encoding = 'utf-8') 
# record = pd.read_csv("credit_record.csv", encoding = 'utf-8') 

In [25]:
data = spark.read.csv('Archive 2/application_record.csv',inferSchema=True, header=True)

In [26]:
record = spark.read.csv('Archive 2/credit_record.csv',inferSchema=True, header=True)

In [231]:
record.show()

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|             0|     X|
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|             0|     C|
|5001712|            -1|     C|
|5001712|            -2|     C|
|5001712|            -3|     C|
|5001712|            -4|     C|
|5001712|            -5|     C|
|5001712|            -6|     C|
|5001712|            -7|     C|
|5001712|            -8|     C|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
+-------+--------------+------+
only showing top 20 rows



In [ ]:
# # Ниже, мы для тех, у кого хоть раз были просрчоки больше 60 дней, ставим в таргет 1.
# # # Добавляем срок кредита к параметрам выдачи кредита
# begin_month = pd.DataFrame(record.groupby(["ID"])["MONTHS_BALANCE"].agg(min) * - 1)
# begin_month = begin_month.rename(columns={'MONTHS_BALANCE':'begin_month'}) 
# new_data = pd.merge(data, begin_month,how="left",on="ID") 

In [131]:
sdf=record.groupBy('id').min('MONTHS_BALANCE')

### умножим на -1 для получения положительных чисел в 'MONTHS_BALANCE'

In [132]:
sdf=sdf.withColumn('begin_month',sdf['min(MONTHS_BALANCE)'] * -1).drop(sdf['min(MONTHS_BALANCE)'])

In [133]:
sdf.show(5)

+-------+-----------+
|     id|begin_month|
+-------+-----------+
|5001812|         22|
|5001849|          8|
|5001921|         19|
|5003338|         33|
|5003386|          7|
+-------+-----------+
only showing top 5 rows



In [135]:
new_data=data.join(sdf,on='id',how="left")

In [272]:
#data test
#new_data.select('id','begin_month').where(new_data['id']=='5008804').show()
new_data.show(5)

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|begin_month|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+
|5008804|          M|           Y|              Y|           0|        427500.0|             Working|    Higher education|      Civil marriage| Rented apartment

In [ ]:
# # # Больше 60, то это просрочка, ставим - Yes, если просрочка есть за срок кредита,то так же ставим Yes
# record['dep_value'] = None
# record['dep_value'][record['STATUS'] == '2'] = 'Yes'
# record['dep_value'][record['STATUS'] == '3'] = 'Yes'
# record['dep_value'][record['STATUS'] == '4'] = 'Yes'
# record['dep_value'][record['STATUS'] == '5'] = 'Yes'
# cpunt = record.groupby('ID').count()
# cpunt['dep_value'][cpunt['dep_value'] > 0] = 'Yes' 
# cpunt['dep_value'][cpunt['dep_value'] == 0] = 'No'

### сделаем фильтрацию по полю STATUS: string

In [262]:
cpunt=record.filter("STATUS=='0' or STATUS=='2' or STATUS=='3' or STATUS=='4' or STATUS=='5'")

In [244]:
# record.filter("STATUS=='0' or STATUS=='2' or STATUS=='3' or STATUS=='4' or STATUS=='5'").groupby('ID').count()

### проверим значения, которые попали в колонку 'STATUS' - от 0 до 5

In [245]:
cpunt.describe().show()

+-------+-----------------+-------------------+-------------------+
|summary|               ID|     MONTHS_BALANCE|             STATUS|
+-------+-----------------+-------------------+-------------------+
|  count|           386224|             386224|             386224|
|   mean|5068504.870748581|-21.842368677244295|0.03120727867765856|
| stddev|45859.31637436183| 15.040049897593708|0.36646649920086827|
|    min|          5001711|                -60|                  0|
|    max|          5150485|                  0|                  5|
+-------+-----------------+-------------------+-------------------+



### проверим значения, которые попали в колонку 'STATUS' - от 0 до 5 и тут не должно быть '1'

In [246]:
cpunt.filter(cpunt["STATUS"]=='1').show()

+---+--------------+------+
| ID|MONTHS_BALANCE|STATUS|
+---+--------------+------+
+---+--------------+------+



In [247]:
# from pyspark.sql import Window
from pyspark.sql import functions as F

In [248]:
# windSpec=Window()\
# .partitionBy('ID')\
# .orderBy('STATUS')

In [250]:
# cpunt=cpunt.groupBy('ID').count()

In [263]:
cpunt.show()

+-------+--------------+------+
|     ID|MONTHS_BALANCE|STATUS|
+-------+--------------+------+
|5001711|            -1|     0|
|5001711|            -2|     0|
|5001711|            -3|     0|
|5001712|            -9|     0|
|5001712|           -10|     0|
|5001712|           -11|     0|
|5001712|           -12|     0|
|5001712|           -13|     0|
|5001712|           -14|     0|
|5001712|           -15|     0|
|5001712|           -16|     0|
|5001712|           -17|     0|
|5001712|           -18|     0|
|5001717|            -5|     0|
|5001717|            -6|     0|
|5001717|            -7|     0|
|5001717|            -8|     0|
|5001717|            -9|     0|
|5001717|           -10|     0|
|5001717|           -11|     0|
+-------+--------------+------+
only showing top 20 rows



In [266]:
cpunt=cpunt.select('ID','STATUS',F\
             .when(cpunt['STATUS']>0,'Yes')\
             .when(cpunt['STATUS']==0,'No')\
             .otherwise('no_data')\
             .alias('dep_value'))

### посмотрим на новую колонку и данные в ней:

In [418]:
cpunt.describe().show()

+-------+-----------------+---------+
|summary|               ID|dep_value|
+-------+-----------------+---------+
|  count|           386224|   386224|
|   mean|5068504.870748581|     null|
| stddev|45859.31637436183|     null|
|    min|          5001711|       No|
|    max|          5150485|      Yes|
+-------+-----------------+---------+



In [275]:
cpunt=cpunt.drop('STATUS')

In [ ]:
# # # Джойним всё вместе,заменяем Yes и No на 1 и 0
# cpunt = cpunt[['dep_value']]
# new_data = pd.merge(new_data, cpunt, how='inner', on='ID')
# new_data['target'] = new_data['dep_value']
# new_data.loc[new_data['target'] == 'Yes','target'] = 1
# new_data.loc[new_data['target'] == 'No','target'] = 0

In [281]:
new_data=new_data.join(cpunt,on='ID',how="inner")

In [304]:
new_data.show(2,vertical=True)

-RECORD 0-----------------------------------
 ID                  | 5008806              
 CODE_GENDER         | M                    
 FLAG_OWN_CAR        | Y                    
 FLAG_OWN_REALTY     | Y                    
 CNT_CHILDREN        | 0                    
 AMT_INCOME_TOTAL    | 112500.0             
 NAME_INCOME_TYPE    | Working              
 NAME_EDUCATION_TYPE | Secondary / secon... 
 NAME_FAMILY_STATUS  | Married              
 NAME_HOUSING_TYPE   | House / apartment    
 DAYS_BIRTH          | -21474               
 DAYS_EMPLOYED       | -1134                
 FLAG_MOBIL          | 1                    
 FLAG_WORK_PHONE     | 0                    
 FLAG_PHONE          | 0                    
 FLAG_EMAIL          | 0                    
 OCCUPATION_TYPE     | Security staff       
 CNT_FAM_MEMBERS     | 2.0                  
 begin_month         | 29                   
 dep_value           | No                   
 target              | 0                    
-RECORD 1-

In [290]:
new_data=new_data.select('*',F\
             .when(new_data['dep_value']=='Yes',1)\
             .when(new_data['dep_value']=='No',0)\
             .otherwise('no_data')\
             .alias('target'))

In [ ]:
# #  В итоге к анкетным данным мы добавили таргет
# new_data.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,begin_month,dep_value,target
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,15.0,No,0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0,14.0,No,0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0,29.0,No,0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,4.0,No,0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0,26.0,No,0


In [303]:
new_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- FLAG_MOBIL: integer (nullable = true)
 |-- FLAG_WORK_PHONE: integer (nullable = true)
 |-- FLAG_PHONE: integer (nullable = true)
 |-- FLAG_EMAIL: integer (nullable = true)
 |-- OCCUPATION_TYPE: string (nullable = true)
 |-- CNT_FAM_MEMBERS: double (nullable = true)
 |-- begin_month: integer (nullable = true)
 |-- dep_value: string (nullable = false)
 |-- target: integer (nullable = true)



сменим таргет на int

In [302]:
new_data=new_data.withColumn("target", new_data['target'].cast('int'))

In [308]:
# # Упростим себе задачу и оставим только часть признаков
# features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']	
# target = ['target',]
# dataset = new_data[features + target]
# dataset[target[0]] = pd.to_numeric(dataset[target[0]])

### переведем категориальные фичи в численные

In [307]:
features = ['AMT_INCOME_TOTAL', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN']

In [315]:
from pyspark.ml.feature import StringIndexer

In [360]:
indexer=StringIndexer(inputCols=['CODE_GENDER','FLAG_OWN_CAR', 'FLAG_OWN_REALTY'],outputCols=['CODE_GENDER_index','FLAG_OWN_CAR_index', 'FLAG_OWN_REALTY_index'])

In [369]:
new_features=['CODE_GENDER_index','FLAG_OWN_CAR_index', 'FLAG_OWN_REALTY_index']

In [372]:
dataset=indexer.fit(new_data).transform(new_data)

### добавим Таргет

In [367]:
dataset.show()

+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+---------+------+-----------------+------------------+---------------------+
|     ID|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|FLAG_MOBIL|FLAG_WORK_PHONE|FLAG_PHONE|FLAG_EMAIL|OCCUPATION_TYPE|CNT_FAM_MEMBERS|begin_month|dep_value|target|CODE_GENDER_index|FLAG_OWN_CAR_index|FLAG_OWN_REALTY_index|
+-------+-----------+------------+---------------+------------+----------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+----------+---------------+----------+----------+---------------+---------------+-----------+---------

In [373]:
dataset=dataset.select(new_features+['target','AMT_INCOME_TOTAL','CNT_CHILDREN'])

### посмотрим что вышло

In [374]:
dataset.show()

+-----------------+------------------+---------------------+------+----------------+------------+
|CODE_GENDER_index|FLAG_OWN_CAR_index|FLAG_OWN_REALTY_index|target|AMT_INCOME_TOTAL|CNT_CHILDREN|
+-----------------+------------------+---------------------+------+----------------+------------+
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              1.0|               1.0|                  0.0|     0|        112500.0|           0|
|              0.0| 

In [306]:
# from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler

In [378]:
faetureassembler=VectorAssembler(inputCols=new_features+['AMT_INCOME_TOTAL','CNT_CHILDREN'],outputCol='ind_features')

In [379]:
output=faetureassembler.transform(dataset)

посмотрим список фичей

In [386]:
output.select('ind_features').show(truncate=False)

+--------------------------+
|ind_features              |
+--------------------------+
|[1.0,1.0,0.0,427500.0,0.0]|
|[1.0,1.0,0.0,427500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|[1.0,1.0,0.0,112500.0,0.0]|
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
|(5,[3],[270000.0])        |
+--------------------------+
only showing top 20 rows



In [389]:
finalized_data=output.select('ind_features','target')

In [390]:
finalized_data.show()

+--------------------+------+
|        ind_features|target|
+--------------------+------+
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|[1.0,1.0,0.0,1125...|     0|
|  (5,[3],[270000.0])|     0|
|  (5,[3],[270000.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
|  (5,[3],[283500.0])|     0|
+--------------------+------+
only showing top 20 rows



У нас есть выборка, где указаны параметры клиента, и вышел ли он на просрочку или нет.

In [ ]:
# # Разделим выборку на трейн и тест, на трейн будем обучать модель, на тест валидировать.
# X_train, X_test, y_train, y_test = train_test_split(dataset[features], pd.to_numeric(dataset[target[0]]), test_size=0.3, random_state=42)

In [391]:
train_data,test_data=finalized_data.randomSplit([0.75,0.25])

In [ ]:
# # Создадим простейшую модель, которая покажет через линейные коэффиценты связь переменных и таргета
# model = LogisticRegression()
# model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
# # Отскалируем численные
# mms = MinMaxScaler()
# mms.fit(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
# X_train_scaled = mms.transform(X_train[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])
# X_test_scaled = mms.transform(X_test[['AMT_INCOME_TOTAL', 'CNT_CHILDREN']])

# X_train_scaled = pd.DataFrame(X_train_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])
# X_test_scaled = pd.DataFrame(X_test_scaled, columns=['AMT_INCOME_TOTAL', 'CNT_CHILDREN'])

#  Модель

In [392]:
from pyspark.ml.classification import LogisticRegression

In [397]:
log_reg=LogisticRegression(featuresCol='ind_features', labelCol='target')

In [398]:
log_reg=log_reg.fit(train_data)

In [413]:
log_reg.coefficients

DenseVector([0.4709, -0.124, 0.0144, -0.0, -0.03])

In [414]:
pred_test=log_reg.evaluate(test_data)

In [415]:
pred_train=log_reg.evaluate(train_data)

### смотрим предсказание модели

In [419]:
pred_train.predictions.describe().show()

+-------+--------------------+----------+
|summary|              target|prediction|
+-------+--------------------+----------+
|  count|              220247|    220247|
|   mean|0.009566532120755334|       0.0|
| stddev| 0.09733990242483842|       0.0|
|    min|                   0|       0.0|
|    max|                   1|       0.0|
+-------+--------------------+----------+



In [417]:
print(f'Точность модели на трейне {pred_train.accuracy}, на тесте {pred_test.accuracy}')

Точность модели на трейне 0.9904334678792447, на тесте 0.9901549805420905


In [ ]:
# # Превращаем категориальные факторы в численные
# ohe = OneHotEncoder()
# ohe.fit(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
# X_train_ohe = ohe.transform(X_train[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])
# X_test_ohe = ohe.transform(X_test[['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']])

# X_train_ohe = pd.DataFrame(X_train_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])
# X_test_ohe = pd.DataFrame(X_test_ohe.toarray(), columns=[item for sublist in ohe.categories_ for item in sublist])

In [ ]:
# X_train = pd.concat([X_train_scaled, X_train_ohe,], axis=1)
# X_test = pd.concat([X_test_scaled, X_test_ohe, ], axis=1)

In [ ]:
# train_score, test_score = accuracy_score(model.predict(X_train), y_train), accuracy_score(model.predict(X_test), y_test)
# print(f'Точность модели на трейне {train_score}, на тесте {test_score}')

Точность модели на трейне 0.9828755045260394, на тесте 0.983635033827025
